In [ ]:
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
import glob, ntpath

In [ ]:
def file_modify(filename, a=None):
    with open(filename, 'r+') as file:
        content = file.read()
        for ch in ['"']:
            if ch in content:
                content = content.replace(ch,'')
    with open(filename, 'w') as file:
        file.write(content)
        
df = pd.read_csv("output/functions.csv")
cols = ['TYPE', 'ASSOC. WITH']
df = df[cols]
df = df[df['ASSOC. WITH'].notnull()]
df.reset_index(drop=True, inplace=True)
a = df['ASSOC. WITH'].unique()
df = pd.DataFrame(a,columns=['Dialog'])
df['Dialog'] = "     DIALOG=(" + df['Dialog'] + ")"
df['Version'] = 'VER=0001'
df['Reports'] = 'REPORTS=(SUMMARY,PROCESSES)'
df.to_csv("1.txt", index=False, header=False)
file_modify("1.txt")
        

In [ ]:
text = "C:/Users/TapajyotiDeb/Downloads/mrdp/MRDPADSC.TXT"

with open(text,'r') as file:
    content = file.read()
    lines = content.split("DIALOG...: ")
    lines = [x for x in lines if x]

dialogname = []
schema = []
onmap = []
inmap = []
oumap = []
with open(text,'r') as file:
    content = file.readlines()
    for line in content:
        if "DIALOG...:" in line:
            try:
                name = re.search('DIALOG...: (.*) SCHEMA', line).group(1)
                dialogname.append(name.strip())
            except:
                dialogname.append('')
        if "SCHEMA...:" in line:
            try:
                name = re.search('SCHEMA...: (.*) ONLINE', line).group(1)
                schema.append(name.strip())
            except:
                schema.append('')
        if "ONLINE MAP:" in line:
            try:
                name = re.search('ONLINE MAP: (.*) INPUT', line).group(1)
                onmap.append(name.strip())
            except:
                onmap.append('')
        if "INPUT MAP.:" in line:
            try:
                name = re.search('INPUT MAP.: (.*) OUTPUT', line).group(1)
                inmap.append(name.strip())
            except:
                inmap.append('')
        if "OUTPUT MAP" in line:
            try:
                name = re.search('OUTPUT MAP:(.*)', line).group(1)
                oumap.append(name.strip())
            except:
                oumap.append('')
                
df1 = pd.DataFrame(list(zip(dialogname, schema, onmap, inmap, oumap)), columns=['DIALOG', 'SCHEMA', 'ONLINE MAP', 'INPUT MAP', 'OUTPUT MAP'])
df1.to_csv("output/adsc_dialog.csv", index=False)

In [ ]:
outname = ntpath.basename(text).split(".")[0]
newdf =df1[['ONLINE MAP']]
newdf.replace('', np.nan, inplace=True)
newdf.dropna(inplace=True)
newdf.reset_index(drop=True, inplace=True)
newdf = "     MAP=" + newdf + ",VERSION=0001"
newdf.to_csv(outname+"MAPS.txt", index=False, header=False)
file_modify(outname+"MAPS.txt")

In [ ]:
with open(text,'r') as file:
    content = file.read()
    lines1 = content.split("DIALOG...: ")
    lines1 = [x for x in lines1 if x]

diname1 = []    
diname2 = []
diname3 = []
for i in range(len(lines1)):
    dialog = lines1[i].split("\n")
    dname = dialog[0].split()[0]

    for line1 in dialog:
        if "PREMAP PROCESS:" in line1:
            try:
                diname1.append(dname)
            except:
                diname1.append("")
    for line1 in dialog:
        if "INCLUDED MODULE:" in line1:
            try:
                diname2.append(dname)
            except:
                diname2.append("")
    for line1 in dialog:
        if "RESPONSE PROCESS:" in line1:
            try:
                diname3.append(dname)
            except:
                diname3.append("")
pre = pd.DataFrame(diname1, columns=['DIALOG'])
inc = pd.DataFrame(diname2, columns=['DIALOG'])
rsp = pd.DataFrame(diname3, columns=['DIALOG'])

In [ ]:
df2 = pd.DataFrame()
for i in range(len(lines)):
    rec= []
    vers= []
    rtype = []
    dialog = lines[i].split("\n")
    dname = dialog[0].split()[0]
    for line in dialog:
        if "RECORD...:" in line:
            name = re.search('RECORD...: (.*)', line).group(1)
            rec.append(name.split()[0])
            vers.append(name.split()[2])
            rtype.append(name.split()[3])
    df21 = pd.DataFrame(list(zip(rec, vers, rtype)), columns=['RECORD NAME', 'VERSION', 'RECORD TYPE'])
    df21.insert(0,'DIALOG','')
    df21['DIALOG'] = dname
    df2 = pd.concat([df2, df21])
df2.to_csv("output/adsc_records.csv", index=False)

In [ ]:
df_cbl = " "*12 + "COPY IDMS " + df2[['RECORD NAME']]
df_cbl.drop_duplicates(inplace=True)
df_cbl.to_csv("adsc_records.cbl", index=False, header=False)

In [ ]:
module = []
mod_name = []
vers = []
execute = []
event = []
desc = []
for line in content:
    try:
        name1 = re.search("EXECUTE ON EDIT ERRORS:(.*) CONTROL", line).group(1)
    except:
        pass
    try:
        name2 = re.search("CONTROL KEY/EVENT:(.*)", line).group(1)
    except:
        pass
        
    if "PREMAP PROCESS:" in line:
        try:
            v = re.search("VERS.:(.*)", line).group(1)
            name = re.search('PREMAP PROCESS:(.*) VERS', line).group(1)
            module.append("PREMAP PROCESS")
            mod_name.append(name.strip())
            vers.append(v.strip())
            continue
        except:
            mod_name.append("")
            vers.append("")
            continue
    if "INCLUDED MODULE:" in line:
        try:
            v = re.search("VERS.:(.*)", line).group(1)
            name = re.search('INCLUDED MODULE:(.*) VERS', line).group(1)
            module.append("INCLUDED MODULE")
            mod_name.append(name.strip())
            vers.append(v.strip())
            continue
        except:
            mod_name.append("")
            vers.append("")
            continue
    if "MODULE DESCRIPTION:" in line:
        try:
            name = re.search("MODULE DESCRIPTION:(.*)", line).group(1)
            desc.append(name)
            continue
        except:
            desc.append("")
            continue
    

df31 = pd.DataFrame(list(zip(module, mod_name, vers)), columns=['MODULE', 'NAME', 'VERSION'])
df31

In [ ]:
with open(text,'r') as file:
    content = file.read()
    lines1 = content.split("DIALOG...: ")
    lines1 = [x for x in lines1 if x]

diname = []    
for i in range(len(lines1)):
    dialog = lines1[i].split("\n")
    dname = dialog[0].split()[0]

    for line1 in dialog:
        if "PREMAP PROCESS:" in line1:
            try:
                diname.append(dname)
            except:
                diname.append("")
        if "INCLUDED MODULE:" in line1:
            try:
                diname.append(dname)
            except:
                diname.append("")
aa = pd.DataFrame(diname, columns=['DIALOG'])
df31 = pd.concat([aa, df31], axis=1)
df31

In [ ]:
module = []
mod_name = []
vers = []
execute = []
event = []
desc = []
for line in content:
    if "RESPONSE PROCESS:" in line:
        try:
            v = re.search("VERS.:(.*)", line).group(1)
            name = re.search('RESPONSE PROCESS:(.*) VERS', line).group(1)
            module.append("RESPONSE PROCESS")
            mod_name.append(name.strip())
            vers.append(v.strip())
            continue
        except:
            mod_name.append("")
            vers.append("")
            continue
    if "EXECUTE ON EDIT ERRORS" in line:
        try:
            name1 = re.search("EXECUTE ON EDIT ERRORS:(.*) CONTROL", line).group(1)
            name2 = re.search("CONTROL KEY/EVENT:(.*)", line).group(1)
            execute.append(name1.strip())
            event.append(name2.strip())
            continue
        except:
            execute.append("")
            event.append("")
            continue
    if "MODULE DESCRIPTION:" in line:
        try:
            name = re.search("MODULE DESCRIPTION:(.*)", line).group(1)
            desc.append(name)
            continue
        except:
            desc.append("")
            continue

In [ ]:
with open(text,'r') as file:
    content = file.readlines()
counter = 0
for line in content:
    counter += 1
    if "RESPONSE PROCESS:" in line:  
        break
s = ''
for line in content[counter-1:]:
    s += line

In [ ]:
lines = s.split("RESPONSE PROCESS: ")
code = []
for line in lines[1:]:
    if "CODE:" in line:
        name = re.search("CODE: (.*)\n",line).group(1)
        code.append(name)
    elif "CODE:" not in line:
        code.append("")

In [ ]:
df32 = pd.DataFrame(list(zip(module, mod_name, vers, execute, event, code)), columns=['MODULE', 'NAME', 'VERSION', 'EXECUTE ON EDIT ERRORS', 'CONTROL KEY/EVENT', 'CODE'])
df32

In [ ]:
df32 = pd.concat([rsp, df32], axis=1)
df32

In [ ]:
a = pd.concat([df31, df32])
a.reset_index(drop=True, inplace=True)
df3 = a.replace(np.nan, '', regex=True)
df3.to_csv("output2/processes.csv", index=False)

In [ ]:
with open(text,'r') as file:
    content = file.readlines()
counter = 0

In [ ]:
for line in content:
    counter += 1
    if "RESPONSE PROCESS:" in line:  
        break
s = ''
for line in content[counter-1:]:
    s += line
lines = s.split("RESPONSE PROCESS: ")

In [ ]:
def code_logic(lines):
    code = []
    for i in range(len(lines)-1):
        a = lines[i:]
        b = a[1].split("\n")
        counter = 0
        for item in b:
            counter += 1
            if "DATE LAST UPDATED:" in item:
                break
        c = b[counter:]
        if c == []:
            code.append("")
            continue
        buf = ''
        count = 0
        for item in range(len(c)):
            try:
                if ((c[item].split()[0] == '0') and ('000' in c[item].split()[1])) or ('000' in c[item].split()[0]):
                    if c[item].split()[1] == '00000100':
                        count += 1
                        if count > 1:
                            break
                    buf += c[item] + "\n"
            except:
                continue
        code.append(buf)
    return code
logic = code_logic(lines)

In [ ]:
df41 = df32[['MODULE', 'NAME']]
df42 = pd.DataFrame(logic, columns=['PROCESS/MODULE CODE'])
df43 = pd.concat([rsp, df41, df42], axis=1)
df43

In [ ]:
counter = 0
for line in content:
    counter += 1
    if "PREMAP PROCESS:" in line:  
        break
s = ''
for line in content[counter-1:]:
    s += line
lines = s.split("PREMAP PROCESS: ")

In [ ]:
logic = code_logic(lines)

In [ ]:
df411 = df31[df31['MODULE'] == 'PREMAP PROCESS']
df411 = df411[['MODULE', 'NAME']].reset_index(drop=True)
df421 = pd.DataFrame(logic, columns=['PROCESS/MODULE CODE'])
df44 = pd.concat([pre, df411, df421], axis=1)
df44

In [ ]:
counter = 0
for line in content:
    counter += 1
    if "INCLUDED MODULE:" in line:  
        break
s = ''
for line in content[counter-1:]:
    s += line
lines = s.split("INCLUDED MODULE: ")

In [ ]:
logic = code_logic(lines)

In [ ]:
df412 = df31[df31['MODULE'] == 'INCLUDED MODULE']
df412 = df412[['MODULE', 'NAME']].reset_index(drop=True)
df422 = pd.DataFrame(logic, columns=['PROCESS/MODULE CODE'])
df45 = pd.concat([inc, df412, df422], axis=1)
df45

In [ ]:
df4 = pd.concat([df43, df44, df45])
df4.to_csv("output2/module_code.csv", index=False)

In [ ]:
with open(text,'r') as file:
    content = file.read()
    lines = content.split("DIALOG...: ")
    lines = [x for x in lines if x]

diname = []    
for i in range(len(lines)):
    dialog = lines[i].split("\n")
    dname = dialog[0].split()[0]

    for line in dialog:
        if "RESPONSE PROCESS:" in line:
            try:
                diname.append(dname)
                continue
            except:
                diname.append("")

In [ ]:
aa = pd.DataFrame(diname, columns=['DIALOG'])
aa